In [1]:
import os, sys
project_path = '/content/gdrive/MyDrive/master/2.5D_sketches_estimator'
sys.path.append(project_path)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd {project_path}

/content/gdrive/MyDrive/master/2.5D_sketches_estimator


In [3]:
# import DL modules
import tensorflow as tf
import keras as keras
from keras.models import Sequential, Model
from keras import layers
from keras import optimizers
from keras.utils.vis_utils import plot_model

# import service modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

In [4]:
#@title Setup
# network and training parameters
RANDOM_SEED = 10
SAMPLE_RATE = 0.005 #@param
N_EPOCH = 50 #@param {type: "integer"}
BATCH_SIZE = 32 #@param {type: "integer"}
VERBOSE = 1 #@param {type: "boolean"}
# LOSS_FUNCTION = 'MeanSquaredError'
LOSS_FUNCTION = 'SSIM'
VALIDATION_SPLIT = 0.2 #@param
TEST_SPLIT = 0.1 #@param
INPUT_SHAPE = (256, 256, 256, 3) #@param
HEIGHT = INPUT_SHAPE[1]
WIDTH = INPUT_SHAPE[2]
LR = 0.0002
OPTIMIZER = tf.optimizers.Adam(
  learning_rate=LR,
  amsgrad=False
)

# Load Data

In [5]:
# reference: https://keras.io/examples/vision/depth_estimation/
class DataLoader(tf.keras.utils.Sequence):
  def __init__(self, data, batch_size=6, dim=(256, 256), n_channels=3, shuffle=True):
    """
    Initialization
    """
    self.data = data
    self.indices = self.data.index.tolist()
    self.dim = dim
    self.n_channels = n_channels
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.min_depth = 0.1
    self.on_epoch_end()

  def __len__(self):
    return int(np.ceil(len(self.data) / self.batch_size))

  def __getitem__(self, index):
    # modify batch size of last batch
    if (index + 1) * self.batch_size > len(self.indices):
      self.batch_size = len(self.indices) - index * self.batch_size
    # Generate one batch of data
    # Generate indices of the batch
    index = self.indices[index * self.batch_size : (index + 1) * self.batch_size]
    # Find list of IDs
    batch = [self.indices[k] for k in index]
    x, y = self.load_batch(batch)
    return x, y


  def on_epoch_end(self):
    """
    Updates indexes after each epoch
    """
    self.index = np.arange(len(self.indices))
    if self.shuffle == True:
      np.random.shuffle(self.index)

  def load(self, image_path, sketch_path):
    """
    Load image and 2.5D sketch pair.
    """
    img = cv2.imread(image_path)
    img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img = np.float16(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    sketch = cv2.imread(sketch_path)
    sketch = cv2.normalize(sketch, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    sketch = np.float16(cv2.cvtColor(sketch, cv2.COLOR_BGR2RGB))
    return img, sketch

  def load_batch(self, batch):
    """
    Load one batch of data.
    """
    x = np.empty((self.batch_size, *self.dim, self.n_channels), dtype='float16')
    y = np.empty((self.batch_size, *self.dim, self.n_channels), dtype='float16')

    for i, batch_id in enumerate(batch):
      x[i,], y[i,] = self.load(
        self.data["raw_image_file_paths"][batch_id],
        self.data["normal_masked_paths"][batch_id],
      )
    return x, y

In [6]:
index_df = pd.read_csv(os.getcwd()+'/data_paths.csv')
index_df = index_df.sample(frac=SAMPLE_RATE, random_state=RANDOM_SEED, ignore_index=True)
num_instances = len(index_df.index)
validation_boundary = int(num_instances*VALIDATION_SPLIT)
test_boundary = int(num_instances*TEST_SPLIT) + validation_boundary

# create data loaders
train_loader = DataLoader(data=index_df[test_boundary:].reset_index(drop='true'), batch_size=BATCH_SIZE, dim=INPUT_SHAPE[1:3], n_channels=INPUT_SHAPE[3], shuffle=False)
test_loader = DataLoader(data=index_df[validation_boundary:test_boundary].reset_index(drop='true'), batch_size=BATCH_SIZE, dim=INPUT_SHAPE[1:3], n_channels=INPUT_SHAPE[3], shuffle=False)
validation_loader = DataLoader(data=index_df[:validation_boundary].reset_index(drop='true'), batch_size=BATCH_SIZE, dim=INPUT_SHAPE[1:3], n_channels=INPUT_SHAPE[3], shuffle=False)

In [7]:
samples = next(iter(DataLoader(data=index_df, batch_size=4)))

error: ignored

In [ ]:
img = np.float32(samples[1][1])
from matplotlib.pyplot import imshow
imshow(img)

In [ ]:
print(len(train_loader.index))
print(len(validation_loader.index))
print(len(test_loader.index))

# Define Model

In [ ]:
class EncodeBlock(layers.Layer):
  def __init__(self, filters, kernel_size, strides):
    super().__init__()
    self.conv1 = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer = 'he_normal')
    self.relu1 = layers.LeakyReLU(alpha=0.2)
    self.bn1 = layers.BatchNormalization()

    self.conv2 = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer = 'he_normal')
    self.relu2 = layers.LeakyReLU(alpha=0.2)
    self.bn2 = layers.BatchNormalization()

    self.pool = layers.MaxPool2D(pool_size=(2, 2))
  def call(self, inputs):
    r2 = self.conv1(inputs)
    r2 = self.bn1(r2)
    r2 = self.relu1(r2)

    r2 = self.conv2(r2)
    r2 = self.bn2(r2)
    r2 = self.relu2(r2)

    r1 = self.pool(r2)
    return r1, r2

class DecodeBlock(layers.Layer):
  def __init__(self, filters, kernel_size, strides):
    super().__init__()
    self.us = layers.UpSampling2D(size=(2, 2))

    self.concat = layers.Concatenate()

    self.conv1 = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer = 'he_normal')
    self.relu1 = layers.LeakyReLU(alpha=0.2)
    self.bn1 = layers.BatchNormalization()

    self.conv2 = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer = 'he_normal')
    self.relu2 = layers.LeakyReLU(alpha=0.2)
    self.bn2 = layers.BatchNormalization()
  def call(self, inputs, passed_inputs):
    r = self.us(inputs)
    r = self.concat([r, passed_inputs])

    r = self.conv1(r)
    r = self.bn1(r)
    r = self.relu1(r)

    r = self.conv2(r)
    r = self.bn2(r)
    r = self.relu2(r)

    return r

class BottleNeckBlock(layers.Layer):
  def __init__(self, filters, kernel_size, strides):
    super().__init__()
    self.drop = layers.Dropout(rate=0.4)
    self.conv1 = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer = 'he_normal')
    self.relu1 = layers.LeakyReLU(alpha=0.2)

    self.conv2 = layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same', kernel_initializer = 'he_normal')
    self.relu2 = layers.LeakyReLU(alpha=0.2)
  def call(self, inputs):
    r = self.drop(inputs)
    r = self.conv1(r)
    r = self.relu1(r)

    r = self.conv2(r)
    r = self.relu2(r)
    return r

In [ ]:
class SketchEstimator(keras.Model):
  def __init__(self, kernel_size_encoder, strides_encoder, 
            kernel_size_bottle_neck, strides_bottle_neck,
            kernel_size_decoder, strides_decoder
            ):
    super().__init__()
    layers_filters = [16, 32, 64, 128, 256]
    # layers_filters = [32, 64, 128, 256, 256]
    self.encoder = [
      EncodeBlock(filters=layers_filters[0], kernel_size=kernel_size_encoder, strides=strides_encoder),
      EncodeBlock(filters=layers_filters[1], kernel_size=kernel_size_encoder, strides=strides_encoder),
      EncodeBlock(filters=layers_filters[2], kernel_size=kernel_size_encoder, strides=strides_encoder),
      EncodeBlock(filters=layers_filters[3], kernel_size=kernel_size_encoder, strides=strides_encoder),
    ]
    self.bottle_neck = BottleNeckBlock(layers_filters[4], kernel_size_bottle_neck, strides_bottle_neck)
    self.decoder = [
      DecodeBlock(filters=layers_filters[3], kernel_size=kernel_size_decoder, strides=strides_decoder),
      DecodeBlock(filters=layers_filters[2], kernel_size=kernel_size_decoder, strides=strides_decoder),
      DecodeBlock(filters=layers_filters[1], kernel_size=kernel_size_decoder, strides=strides_decoder),
      DecodeBlock(filters=layers_filters[0], kernel_size=kernel_size_decoder, strides=strides_decoder),
    ]
    self.out = layers.Conv2D(filters=3, kernel_size=3, strides=1, padding='same')
  def call(self, inputs):
    e01, e02 = self.encoder[0](inputs)
    e11, e12 = self.encoder[1](e01)
    e21, e22 = self.encoder[2](e11)
    e31, e32 = self.encoder[3](e21)

    b = self.bottle_neck(e31)
    
    d = self.decoder[0](b, e32)
    d = self.decoder[1](d, e22)
    d = self.decoder[2](d, e12)
    d = self.decoder[3](d, e02)

    r = self.out(d)
    return r

In [ ]:
model = SketchEstimator(3, 1, 
            3, 1,
            3, 1)
model.build(input_shape=(None, 256, 256, 3))
model.call(layers.Input((256, 256, 3)))
model.summary()


In [ ]:
# customized loss
def ssim_loss(target, pred):
  # # edges
  # dy_true, dx_true = tf.image.image_gradients(target)
  # dy_pred, dx_pred = tf.image.image_gradients(pred)
  # weights_x = tf.exp(tf.reduce_mean(tf.abs(dx_true)))
  # weights_y = tf.exp(tf.reduce_mean(tf.abs(dy_true)))

  # # smoothness
  # smoothness_x = dx_pred * weights_x
  # smoothness_y = dy_pred * weights_y
  # smoothness_loss = tf.reduce_mean(abs(smoothness_x)) + tf.reduce_mean(abs(smoothness_y))

  # Structural similarity (SSIM) index
  loss = tf.reduce_mean(
    1 - tf.image.ssim(
      target, pred, max_val=1.0, filter_size=7, k1=0.01, k2=0.03
    )
  )

  return loss

In [ ]:
# encoder-decoder
if LOSS_FUNCTION == 'SSIM':
  model.compile(loss=ssim_loss, optimizer=OPTIMIZER, metrics=['accuracy'])
else:
  model.compile(loss=LOSS_FUNCTION, optimizer=OPTIMIZER, metrics=['accuracy'])


# Train Model

In [ ]:
# checkpoints
# additional checkpoint load code and temp directory definition
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

checkpoint_path = project_path+"/intermediate/unet/weights{epoch:02d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# definition of checkpoint parameters file
checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=False, save_freq='epoch')

In [ ]:
# train
history = model.fit(train_loader, batch_size=BATCH_SIZE, epochs=N_EPOCH, verbose=VERBOSE, validation_data=validation_loader, callbacks=[checkpoint])

Epoch 1/50
15/15 [==============================] - ETA: 0s - loss: nan - accuracy: 0.8709 
Epoch 1: saving model to /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights01.ckpt


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights01.ckpt/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights01.ckpt/assets


15/15 [==============================] - 323s 21s/step - loss: nan - accuracy: 0.8709 - val_loss: nan - val_accuracy: 0.8984
Epoch 2/50
15/15 [==============================] - ETA: 0s - loss: nan - accuracy: 0.9028
Epoch 2: saving model to /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights02.ckpt


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights02.ckpt/assets


...

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights49.ckpt/assets


15/15 [==============================] - 10s 660ms/step - loss: nan - accuracy: 0.9041 - val_loss: nan - val_accuracy: 0.8934
Epoch 50/50
15/15 [==============================] - ETA: 0s - loss: nan - accuracy: 0.9051
Epoch 50: saving model to /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights50.ckpt


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights50.ckpt/assets


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/master/2.5D_sketches_estimator/intermediate/unet/weights50.ckpt/assets


15/15 [==============================] - 10s 659ms/step - loss: nan - accuracy: 0.9051 - val_loss: nan - val_accuracy: 0.8934


In [ ]:
# continue train
new_model = load_model(project_path+"/intermediate/weights%02d.ckpt" % history.params['epochs'])
remain_n_epoch = N_EPOCH - history.params['epochs']
if remain_n_epoch == 0:
  print("Train finished.")
else:
  new_history = new_model.fit(train_loader, batch_size=BATCH_SIZE, epochs=remain_n_epoch, verbose=VERBOSE, validation_data=validation_loader, callbacks=[checkpoint])